In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv("D:\OneDrive - NITT\Custom_Download\R_cleaned_data (2).csv")

In [3]:
def target_encode(age,gender):
    if age<=40 and gender=='Male':
        return 'Young men'
    elif age>40 and gender=='Male':
        return 'Old men'
    elif age<=40 and gender=='Female':
        return 'Young women'
    else:
        return 'Old women'

In [4]:
df['Age_normalized'] = df['Age'].apply(lambda x: sum(map(int,(x.split('-'))))/2)
df['Age'] = df.apply(lambda x: target_encode(x.Age_normalized,x.Gender),axis=1)

In [5]:
list(df)[:-9]

['Age',
 'Gender',
 'AMITRIPTYLINE',
 'BUPROPION',
 'CITALOPRAM',
 'DESVENLAFAXINE',
 'DOXEPIN',
 'DULOXETINE',
 'ESCITALOPRAM',
 'FLUOXETINE',
 'MIRTAZAPINE',
 'NORTRIPTYLINE',
 'PAROXETINE',
 'ROPINIROLE',
 'SERTRALINE',
 'TRAZODONE',
 'VENLAFAXINE']

In [16]:
df

,Age,Gender,AMITRIPTYLINE,BUPROPION,CITALOPRAM,DESVENLAFAXINE,DOXEPIN,DULOXETINE,ESCITALOPRAM,FLUOXETINE,MIRTAZAPINE,NORTRIPTYLINE,PAROXETINE,ROPINIROLE,SERTRALINE,TRAZODONE,VENLAFAXINE
0,Young women,Female,AMITRIPTYLINE,BUPROPION,CITALOPRAM,DESVENLAFAXINE,DOXEPIN,DULOXETINE,ESCITALOPRAM,FLUOXETINE,MIRTAZAPINE,NORTRIPTYLINE,PAROXETINE,ROPINIROLE,SERTRALINE,TRAZODONE,VENLAFAXINE
1,Young women,Female,nan,nan,nan,nan,nan,nan,nan,FLUOXETINE,nan,nan,nan,nan,SERTRALINE,nan,nan
2,Young women,Female,nan,BUPROPION,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,SERTRALINE,nan,nan
3,Young women,Female,nan,nan,nan,nan,nan,nan,ESCITALOPRAM,nan,nan,nan,nan,nan,SERTRALINE,nan,nan
4,Young women,Female,nan,nan,CITALOPRAM,nan,nan,nan,nan,nan,nan,nan,nan,nan,SERTRALINE,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16765,Old men,Male,nan,nan,nan,nan,nan,nan,ESCITALOPRAM,nan,nan,nan,nan,nan,nan,nan,nan
16766,Old men,Male,nan,nan,nan,nan,nan,nan,nan,nan,MIRTAZAPINE,nan,nan,nan,nan,TRAZODONE,nan
16767,Old men,Male,nan,nan,nan,nan,nan,DULOXETINE,nan,nan,nan,nan,nan,nan,nan,nan,nan
16768,Old men,Male,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,VENLAFAXINE


In [6]:
df = df[list(df)[:-9]]

In [7]:
for col in list(df)[2:]:
    df[col] = np.where(df[col].isnull(),float('nan'),col)

In [8]:
transactions = []
for i in tqdm(range(len(df))):
    transactions.append([str(df.values[i,j]) for j in range(0, df.shape[1])])

  0%|          | 0/16770 [00:00<?, ?it/s]

In [9]:
transactions_small = transactions[:]

In [10]:
# Training ARM - Apriori on the dataset
from apyori import apriori
rules = apriori(transactions_small, min_support = 0.02, min_confidence = 0.2, min_lift = 3, min_length = 2)

In [11]:
# Visualising the results
results = list(rules)

In [12]:
results

[RelationRecord(items=frozenset({'PAROXETINE', 'DULOXETINE'}), support=0.020691711389385806, ordered_statistics=[OrderedStatistic(items_base=frozenset({'DULOXETINE'}), items_add=frozenset({'PAROXETINE'}), confidence=0.26109857035364936, lift=3.297155892191792), OrderedStatistic(items_base=frozenset({'PAROXETINE'}), items_add=frozenset({'DULOXETINE'}), confidence=0.26129518072289154, lift=3.2971558921917916)]),
 RelationRecord(items=frozenset({'TRAZODONE', 'DULOXETINE'}), support=0.023076923076923078, ordered_statistics=[OrderedStatistic(items_base=frozenset({'DULOXETINE'}), items_add=frozenset({'TRAZODONE'}), confidence=0.29119638826185107, lift=3.240453504413565), OrderedStatistic(items_base=frozenset({'TRAZODONE'}), items_add=frozenset({'DULOXETINE'}), confidence=0.25680159256801594, lift=3.2404535044135647)]),
 RelationRecord(items=frozenset({'DULOXETINE', 'VENLAFAXINE'}), support=0.029278473464519977, ordered_statistics=[OrderedStatistic(items_base=frozenset({'DULOXETINE'}), items_

In [13]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

In [14]:
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,DULOXETINE,PAROXETINE,0.020692,0.261099,3.297156
1,DULOXETINE,TRAZODONE,0.023077,0.291196,3.240454
2,DULOXETINE,VENLAFAXINE,0.029278,0.369451,3.712216
3,Male,Old men,0.180382,0.707437,3.921890
4,Male,Young men,0.074597,0.292563,3.921890
...,...,...,...,...,...
205,Young women,FLUOXETINE,0.021467,0.345489,3.041395
206,BUPROPION,ESCITALOPRAM,0.020632,0.585448,14.146930
207,DULOXETINE,FLUOXETINE,0.020572,0.259594,7.732480
208,ESCITALOPRAM,FLUOXETINE,0.023614,0.439024,10.732418


In [15]:
resultsinDataFrame.nlargest(n = 10, columns = 'Lift')

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
164,BUPROPION,ESCITALOPRAM,0.022063,0.626058,14.622541
206,BUPROPION,ESCITALOPRAM,0.020632,0.585448,14.146930
71,BUPROPION,ESCITALOPRAM,0.023256,0.659898,12.913066
165,BUPROPION,ESCITALOPRAM,0.021825,0.619289,12.467566
75,BUPROPION,FLUOXETINE,0.024925,0.707276,12.202691
167,BUPROPION,FLUOXETINE,0.023494,0.666667,11.793249
72,BUPROPION,ESCITALOPRAM,0.024270,0.688663,11.136821
178,ESCITALOPRAM,FLUOXETINE,0.024508,0.455654,10.900598
208,ESCITALOPRAM,FLUOXETINE,0.023614,0.439024,10.732418
166,BUPROPION,ESCITALOPRAM,0.022838,0.648054,10.728399


- 'BUPROPION' and 'ESCITALOPRAM' will go together to treat the patient